In [79]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/мини-таски/DataCon-2024-Mini-Task_1-main.zip

In [ ]:
path = '/content/DataCon-2024-Mini-Task_1-main/raw_data/UTexas Aptamer Database.xlsx'

In [ ]:
import pandas as pd
df = pd.read_excel(path)
# df.head()
# df.shape

In [ ]:
columns = [' '.join(i.split()) for i in list(df.columns.values)]

In [ ]:
df.columns = columns

In [ ]:
usefull = ['Journals', 'Target', 'Type of Nucleic Acid',
           'Name of Aptamer', 'Aptamer Sequence', 'Sequence Length',
           'GC Content', 'Affinity', 'Kd (nM)', 'Pool Type', 'Pool Random Region',
           'Binding Buffer/Conditions', 'Divalent Salt', 'Type of the buffer',
           'pH', 'Application as quoted in the referenced paper', 'Additional Information']
#  'Additional Information\n\n' придется обрабатывать как текст

useless= ['Year of Paper', 'Link to PubMed Entry', 'Journal DOI', 'Citation',
          'Parent sequence serial number', 'Serial Number'
          'Corresponding Author Name, email address', 'please fill out the form for any feedbacks/comments',
          'Aptagen Cross Referencing(Check  Aptamer Chemistry, Affinity, Length, GC content, sequence)',
          'Post-selex modifications to the aptamer', 'Molecular weight of target'] # большинство колонок (с сайтами) я убрала, так как в дальнейшем они нам не понадобятся для предсказания kd


In [ ]:
df = df[usefull]

In [ ]:
# df[df['Kd (nM)'] == 'TBD'] # -> 1489
# df[df['Pool Random Region'] == 'TBD'] # -> 1488
df = df.drop([1488, 1489])
df['Kd (nM)'] = df['Kd (nM)'].astype('float')

In [ ]:
#df['Pool Random Region'].isna().sum()
df['Pool Random Region'] = df['Pool Random Region'].fillna(0)
df['Pool Random Region'] = df['Pool Random Region'].astype('int')

In [ ]:
# df['Divalent Salt'].isna().sum()
df['Divalent Salt'] = df['Divalent Salt'].fillna('MgCl')
df['Divalent Salt'] = [''.join(i.split()) for i in df['Divalent Salt'].values]

In [ ]:
df['Additional Information'] = df['Additional Information'].fillna('Not reported')

In [ ]:
# df['Type of the buffer'].value_counts()
df['Type of the buffer'] = df['Type of the buffer'].fillna('Tris Buffers')

In [ ]:
mean = df['pH'].mean() # -> round to 7.5
# df['pH'].hist(bins=60) # можно взять среднее для заполнения nan
df['pH'] = df['pH'].fillna(7.5)

In [ ]:
df['Affinity'] = df['Affinity'].str.lower()
df['Affinity'] = [' '.join(i.split()) for i in df['Affinity'].values]

In [ ]:
df.info() # nan -> Kd (nM)

<class 'pandas.core.frame.DataFrame'>
Index: 1493 entries, 0 to 1494
Data columns (total 17 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Journals                                       1493 non-null   object 
 1   Target                                         1493 non-null   object 
 2   Type of Nucleic Acid                           1493 non-null   object 
 3   Name of Aptamer                                1493 non-null   object 
 4   Aptamer Sequence                               1493 non-null   object 
 5   Sequence Length                                1493 non-null   int64  
 6   GC Content                                     1493 non-null   float64
 7   Affinity                                       1493 non-null   object 
 8   Kd (nM)                                        1111 non-null   float64
 9   Pool Type                                      1493 non-n

In [ ]:
df.head()

,Journals,Target,Type of Nucleic Acid,Name of Aptamer,Aptamer Sequence,Sequence Length,GC Content,Affinity,Kd (nM),Pool Type,Pool Random Region,Binding Buffer/Conditions,Divalent Salt,Type of the buffer,pH,Application as quoted in the referenced paper,Additional Information
0,Nature,Cibacron Blue 3GA,ssRNA,CB-42,5'GGGAGAAUUCCCGCGGCAGAAGCCCACCUGGCUUUGAACUCUAU...,132,0.560606,kd~600µm,600000.0,5'-GGGAGAATTCCCGCGG-N98-CTGCAGCCCGGGAAGCTT-3',98,"0.6 ml of 0.5 M LiCI, 20 mM Tris-HCI, pH 7.6, ...",MgCl,Tris Buffers,7.6,"Detection: "" Isolate RNAs that bind to several...",The aptamer was reported in DNA (include thymi...
1,Nature,Reactive Blue 4,ssRNA,B4-25,5'GGGAGAAUUCCCGCGGCGUUGGCCCAGGAUAAUAGGACGAAAUC...,133,0.593985,kd<100µm,100000.0,5'-GGGAGAATTCCCGCGG-N97-CTGCAGCCCGGAAGCTT-3',97,"0.6 ml of 0.5 M LiCI, 20 mM Tris-HCI, pH 7.6, ...",MgCl,Tris Buffers,7.6,"Detection: "" Isolate RNAs that bind to several...",DNA library/pool was used as a template to gen...
2,Science,T4 DNA polymerase (gp43),ssRNA,wild type,5'GAAUUGUGGUGUUGGCUCCCUAUAGUGAGUCGUAUUAAUAUUCC...,113,0.336283,kd: 4.8 nm,4.8,5'-GAATTGTGGTGTTGGCTCCCTATAGTGAGTCGTATTA-ATATT...,9,Not reported,MgCl,Not Reported,7.5,"Detection: "" We have previously shown that the...",DNA library/pool was used as a template to gen...
3,Science,T4 DNA polymerase (gp43),ssRNA,major variant,5'GAAUUGUGGUGUUGGCUCCCUAUAGUGAGUCGUAUUAAUAUUCC...,113,0.353982,kd: 4.8 nm,4.8,5'-GAATTGTGGTGTTGGCTCCCTATAGTGAGTCGTATTA-ATATT...,9,Not reported,MgCl,Not Reported,7.5,"Detection: "" We have previously shown that the...",DNA library/pool was used as a template to gen...
4,Nature,"Thrombin (Sigma), Human",ssDNA,15 mer (colloquially known as Bock DNA Aptamer),5'GGTTGGTGTGGTTGG3',15,0.600000,kd: 25-200 nm,112.5,5'-CGTACGGTCGACGCTAGC-60N-CACGTGGAGCTCGGATCC-3',60,"19 mM Tris-acetate, pH 7.4, 140 mM NaCI, 5 mM ...",MgCl,Tris Buffers,7.4,"Diagnostic and therapeutic: ""We are at present...",Presumed minimized variant was found


In [ ]:
cat_cols = ['Journals', 'Target', 'Type of Nucleic Acid', 'Name of Aptamer',
            'Aptamer Sequence', 'Pool Type', 'Binding Buffer/Conditions',
            'Divalent Salt',	'Type of the buffer',
            'Application as quoted in the referenced paper',
            'Additional Information'] #категориальные фичи

In [ ]:
# df['Sequence Length'].describe() -> max=312
# df[df['Sequence Length'] == 312] # интересный случай, скорее всего это выброс
# df['Sequence Length'].hist(bins=40)

In [ ]:
df.to_csv('version2.csv', index=False)

In [82]:
df = df.drop(0)

In [85]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/мини-таски/data1.xlsx')
df.head()

,version2 (1),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,NaN,Journals,Target,Function,Type of Nucleic Acid,Name of Aptamer,Aptamer Sequence,Sequence Length,GC Content,Affinity,Kd (nM),Pool Type,Pool Random Region,Binding Buffer/Conditions,Divalent Salt,Type of the buffer,pH,Application as quoted in the referenced paper,Additional Information
1,0.0,Nature,Cibacron Blue 3GA,NaN,ssRNA,CB-42,5'GGGAGAAUUCCCGCGGCAGAAGCCCACCUGGCUUUGAACUCUAU...,132,0.5606060606060606,kd~600µm,600000.0,5'-GGGAGAATTCCCGCGG-N98-CTGCAGCCCGGGAAGCTT-3',98,"0.6 ml of 0.5 M LiCI, 20 mM Tris-HCI, pH 7.6, ...",MgCl,Tris Buffers,7.6,"Detection: "" Isolate RNAs that bind to several...",The aptamer was reported in DNA (include thymi...
2,1.0,Nature,Reactive Blue 4,Photoreceptor that mediates primarily blue lig...,ssRNA,B4-25,5'GGGAGAAUUCCCGCGGCGUUGGCCCAGGAUAAUAGGACGAAAUC...,133,0.5939849624060151,kd<100µm,100000.0,5'-GGGAGAATTCCCGCGG-N97-CTGCAGCCCGGAAGCTT-3',97,"0.6 ml of 0.5 M LiCI, 20 mM Tris-HCI, pH 7.6, ...",MgCl,Tris Buffers,7.6,"Detection: "" Isolate RNAs that bind to several...",DNA library/pool was used as a template to gen...
3,2.0,Science,T4 DNA polymerase (gp43),Replicates the viral genomic DNA. This polymer...,ssRNA,wild type,5'GAAUUGUGGUGUUGGCUCCCUAUAGUGAGUCGUAUUAAUAUUCC...,113,0.336283185840708,kd: 4.8 nm,4.8,5'-GAATTGTGGTGTTGGCTCCCTATAGTGAGTCGTATTA-ATATT...,9,Not reported,MgCl,Not Reported,7.5,"Detection: "" We have previously shown that the...",DNA library/pool was used as a template to gen...
4,3.0,Science,T4 DNA polymerase (gp43),Replicates the viral genomic DNA. This polymer...,ssRNA,major variant,5'GAAUUGUGGUGUUGGCUCCCUAUAGUGAGUCGUAUUAAUAUUCC...,113,0.35398230088495575,kd: 4.8 nm,4.8,5'-GAATTGTGGTGTTGGCTCCCTATAGTGAGTCGTATTA-ATATT...,9,Not reported,MgCl,Not Reported,7.5,"Detection: "" We have previously shown that the...",DNA library/pool was used as a template to gen...


In [87]:
columns = df.iloc[0]

In [104]:
columns[0] = '0'

In [105]:
df.columns = columns

In [91]:
df = df.drop(0)

In [107]:
df = df.drop('0', axis=1)

In [110]:
df['Affinity'] = df['Affinity'].fillna(df['Kd (nM)'].median())

In [111]:
df.head()

,Journals,Target,Function,Type of Nucleic Acid,Name of Aptamer,Aptamer Sequence,Sequence Length,GC Content,Affinity,Kd (nM),Pool Type,Pool Random Region,Binding Buffer/Conditions,Divalent Salt,Type of the buffer,pH,Application as quoted in the referenced paper,Additional Information
1,Nature,Cibacron Blue 3GA,NaN,ssRNA,CB-42,5'GGGAGAAUUCCCGCGGCAGAAGCCCACCUGGCUUUGAACUCUAU...,132,0.5606060606060606,kd~600µm,600000.0,5'-GGGAGAATTCCCGCGG-N98-CTGCAGCCCGGGAAGCTT-3',98,"0.6 ml of 0.5 M LiCI, 20 mM Tris-HCI, pH 7.6, ...",MgCl,Tris Buffers,7.6,"Detection: "" Isolate RNAs that bind to several...",The aptamer was reported in DNA (include thymi...
2,Nature,Reactive Blue 4,Photoreceptor that mediates primarily blue lig...,ssRNA,B4-25,5'GGGAGAAUUCCCGCGGCGUUGGCCCAGGAUAAUAGGACGAAAUC...,133,0.5939849624060151,kd<100µm,100000.0,5'-GGGAGAATTCCCGCGG-N97-CTGCAGCCCGGAAGCTT-3',97,"0.6 ml of 0.5 M LiCI, 20 mM Tris-HCI, pH 7.6, ...",MgCl,Tris Buffers,7.6,"Detection: "" Isolate RNAs that bind to several...",DNA library/pool was used as a template to gen...
3,Science,T4 DNA polymerase (gp43),Replicates the viral genomic DNA. This polymer...,ssRNA,wild type,5'GAAUUGUGGUGUUGGCUCCCUAUAGUGAGUCGUAUUAAUAUUCC...,113,0.336283185840708,kd: 4.8 nm,4.8,5'-GAATTGTGGTGTTGGCTCCCTATAGTGAGTCGTATTA-ATATT...,9,Not reported,MgCl,Not Reported,7.5,"Detection: "" We have previously shown that the...",DNA library/pool was used as a template to gen...
4,Science,T4 DNA polymerase (gp43),Replicates the viral genomic DNA. This polymer...,ssRNA,major variant,5'GAAUUGUGGUGUUGGCUCCCUAUAGUGAGUCGUAUUAAUAUUCC...,113,0.35398230088495575,kd: 4.8 nm,4.8,5'-GAATTGTGGTGTTGGCTCCCTATAGTGAGTCGTATTA-ATATT...,9,Not reported,MgCl,Not Reported,7.5,"Detection: "" We have previously shown that the...",DNA library/pool was used as a template to gen...
5,Nature,"Thrombin (Sigma), Human",Component of the adaptor protein complex 2 (AP...,ssDNA,15 mer (colloquially known as Bock DNA Aptamer),5'GGTTGGTGTGGTTGG3',15,0.6,kd: 25-200 nm,112.5,5'-CGTACGGTCGACGCTAGC-60N-CACGTGGAGCTCGGATCC-3',60,"19 mM Tris-acetate, pH 7.4, 140 mM NaCI, 5 mM ...",MgCl,Tris Buffers,7.4,"Diagnostic and therapeutic: ""We are at present...",Presumed minimized variant was found


In [112]:
df.to_csv('version3.csv', index=False)

In [ ]:
import pandas as pd
import requests
import numpy as np


def get_protein_sequence(protein_name):
    protein_name = "+".join(protein_name.split())
    url = f"https://www.uniprot.org/uniprot/?query={protein_name}&format=fasta"
    response = requests.get(url)
    if response.status_code == 200:
        sequence = "".join(response.text.split('\n')[1:])
        return sequence
    else:
        return None



path= './UTexas Aptamer Database.xlsx'
df = pd.read_excel(path)
columns = [' '.join(i.split()) for i in df.columns.to_list()]
df.columns = columns
df['Target'] = df['Target'].transform(lambda x: tuple(x.split(',')))
df.drop_duplicates(subset='Target', keep='first', inplace=True)
df['Target'] = df['Target'].transform(lambda x: ','.join(x))
output_path = 'unique_targets.xlsx'
df.to_excel(output_path, index=False)
for i in df['Target']:
    print(get_protein_sequence(i), i)